In [1]:
import faiss
import numpy as np

## Generate Random Data (Vectors)

In [2]:
num_vectors = 1000
dim = 128


vectors = np.random.random((num_vectors, dim)).astype('float32')

vectors

array([[0.9111588 , 0.00230304, 0.08145921, ..., 0.9097216 , 0.34790474,
        0.44785148],
       [0.8080919 , 0.18920639, 0.51820844, ..., 0.31317604, 0.00619393,
        0.32726598],
       [0.54094696, 0.0862287 , 0.21088399, ..., 0.03956298, 0.8415694 ,
        0.0322605 ],
       ...,
       [0.4897907 , 0.36247817, 0.7902989 , ..., 0.47860926, 0.2825015 ,
        0.18486695],
       [0.9303571 , 0.26425993, 0.4022094 , ..., 0.5873805 , 0.4586839 ,
        0.39962792],
       [0.24969232, 0.80149555, 0.94745684, ..., 0.12798944, 0.8219062 ,
        0.5152512 ]], dtype=float32)

# Create a FAISS Index

An index is essentially a data structure used to efficiently search for the nearest neighbors in the vector space.

Let’s create an IndexFlatL2 index, which uses the L2 distance (Euclidean distance) for nearest neighbor search.

In [9]:
index = faiss.IndexFlatL2(dim)

In [10]:
print(f"IS index Trained : {index.is_trained}")

IS index Trained : True


For more complex searches, FAISS also supports other indexes such as IVF (Inverted File Index) and HNSW (Hierarchical Navigable Small World graphs). But IndexFlatL2 is simple and works well for smaller datasets or testing.

## Add Vectors to the Index

In [11]:
index.add(vectors)

print(f"total number of index : {index.ntotal}")

total number of index : 1000


## Search for Nearest Neighbors

In [14]:
query_vect = np.random.random((1, dim)).astype('float32')

# Find the 5 nearest neighbor
k = 5
distances, indices = index.search(query_vect, k)

print(f'nearest neighbor = {distances}')
print(f'indices : {indices}')

nearest neighbor = [[15.265417 15.52211  15.656538 15.705608 15.858515]]
indices : [[169 982 263 379 135]]


# kmeans

In [17]:
vector = np.random.random((1000, 128)).astype('float32')

k = 3
kmeans = faiss.Kmeans(d=128, k = k, niter=30, verbose=True)
kmeans.train(vector)


print(kmeans.centroids)

[[0.49164623 0.47379702 0.50735193 0.55930084 0.5660486  0.49740916
  0.5279886  0.4755358  0.51397824 0.55433804 0.50143623 0.4975144
  0.5581958  0.50761443 0.46180502 0.53218466 0.54152364 0.50299793
  0.5287189  0.49983895 0.5402887  0.45184365 0.42239    0.48616847
  0.455407   0.5496139  0.54311657 0.57695484 0.51840156 0.4204709
  0.48356608 0.550754   0.5004508  0.5086872  0.46641454 0.4798862
  0.5613355  0.5872064  0.5930706  0.49161938 0.56078947 0.49792632
  0.5512023  0.4826912  0.51173514 0.483661   0.47082013 0.4134657
  0.50804186 0.4968574  0.5437819  0.45483676 0.5265093  0.53929627
  0.5001473  0.5746272  0.5115037  0.43561682 0.5681642  0.5507187
  0.5376441  0.46695668 0.48314974 0.5188913  0.5396886  0.5370281
  0.48142353 0.48756236 0.5131599  0.50367475 0.48451054 0.5308282
  0.46740705 0.45293507 0.5350937  0.48011622 0.48900065 0.4715013
  0.4933333  0.5084192  0.46687022 0.5205309  0.4905006  0.4913374
  0.42434797 0.53138596 0.5154473  0.5490562  0.45467326 

# Product Quantization (PQ) for Memory Efficiency

In [19]:
quatizer = faiss.IndexFlatL2(dim)
pq_index = faiss.IndexIVFPQ(quatizer, dim, 100, 8, 8)

IndexIVFPQ is the actual Product Quantization index you're creating. It combines Inverted File (IVF) and Product Quantization (PQ) for efficient nearest neighbor search.

IVF (Inverted File) is a technique used to partition the dataset into separate cells or clusters, which helps reduce the search space when performing a nearest neighbor search. It works by dividing the dataset into multiple "buckets", so instead of searching the entire dataset, you only need to search within the most relevant bucket (cell). This drastically speeds up the search process.

Product Quantization (PQ) is a technique used to compress the representation of vectors while retaining the ability to perform accurate searches. It works by splitting a vector into smaller subspaces (parts) and quantizing each subspace separately. This reduces the memory required to store the vectors and speeds up the distance calculations during the search.

# Steps of Searching with Product Quantization (PQ)
Splitting the Query Vector:

When you issue a search with a query vector, the query vector is split in the same way as the indexed vectors.
For example, if you have an 8 subquantizer setting, the 128-dimensional query vector will be split into 8 subspaces, each with 16 dimensions.
So, if you are searching with a query vector, it gets split into smaller chunks, just like the vectors in your index.
Quantizing the Query Vector:

Each subspace of the query vector is compared to the codebook that was learned during the training phase.
For each subspace (part of the vector), the algorithm checks which code from the corresponding codebook is the closest to the query subspace. These codebooks are essentially small, predefined representations of each subspace.
So, instead of working with the full 128-dimensional vector directly, the query vector is represented by a set of quantized codes that correspond to the closest matches in each subspace.
Searching for the Closest Match:

Now that the query has been quantized into a set of codes (one code for each subspace), the search begins.
The system compares the query codes with the codes stored in the index.
The indexed vectors are also stored as sets of codes (not as full 128-dimensional vectors) since they were quantized during the training phase.
Distance Calculation (Distance between Codes):

For each query code, the algorithm calculates the distance (usually using Euclidean distance or a similar metric) between the query code and the stored codes in the index.
This step is much faster because you're comparing smaller codes (quantized representations) rather than full high-dimensional vectors.
Retrieving the Data:

Once the closest codes are found, the algorithm retrieves the corresponding full vectors that were stored in the index.
These full vectors are the original, non-compressed vectors (the actual data points) that match the closest codes.
Rank and Return Results:

The search returns the top-k closest vectors to the query vector, based on the distance between their quantized codes.
The final results are ranked by distance, and the corresponding original vectors are returned as the search result.

In [20]:
quatizer = faiss.IndexFlatL2(dim)
pq_index = faiss.IndexIVFPQ(quatizer, dim, 100, 8, 8)

In [21]:
# Train the index (requires at least a sample of your dataset)
pq_index.train(vector)

In [22]:
pq_index.add(vector)

In [23]:
query_vect = np.random.random((1, 128)).astype('float32')

In [24]:
dist, indices = pq_index.search(query_vect, 5)

print(f'nearest neighbors : {dist}')
print(f'indices : {indices}')

nearest neighbors : [[14.387854  14.471326  14.72433   15.08274   15.4168415]]
indices : [[878 805  58  63 278]]
